In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pymorphy2
from gensim.models.keyedvectors import KeyedVectors

morph = pymorphy2.MorphAnalyzer()

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

w2v = KeyedVectors.load_word2vec_format("data/ruwikiruscorpora_0_300_20.bin", binary=True)
w2v.init_sims(True)

w2v_tags = set([w.split('_')[1] for w in w2v.index2word])

In [124]:
file_prefix = 'data/250ktendersList'

with open(file_prefix + '.csv', 'rt', encoding='utf-8') as f:
    content = f.readlines()

content_bad = list()
content_good = list()

def split_line(line):
    line_norm = line
    line_norm = re.sub('\t', ' ', line_norm)
    line_norm = re.sub('\"', '\'', line_norm)
    parts = line_norm.split(';')
    if len(parts) == 7:
        return parts
    parts_new = list()
    for part in parts:
        if parts_new and part.startswith(' '):
            parts_new[-1] = parts_new[-1] + ',' + part
        else:
            parts_new.append(part)
    if len(parts_new) == 7:
        return parts_new
    return None

content_bad.append('\t'.join(content[0].split(';')))
content_good.append('\t'.join(content[0].split(';')))

for line in content[1:]:
    parts = split_line(line)
    if parts is None:
        content_bad.append(line)
        continue
    assert(len(parts) == 7)
    parts[5] = parts[5].ljust(9, "0")
    content_good.append('\t'.join(parts))

with open(file_prefix + '_split.csv', 'wt', encoding='utf-8') as f:
    f.write(''.join(content_good))

with open(file_prefix + '_split_bad.csv', 'wt', encoding='utf-8') as f:
    f.write(''.join(content_bad))


In [125]:
tags_conversion = {
    'ADJF':'ADJ', 'ADJS':'ADJ', 'ADVB':'ADV', 'CONJ':'CCONJ', 
    'INFN':'VERB', 'INTJ':'INTJ', 'NOUN':'NOUN', 'NPRO':'PRON', 
    'NUMR':'NUM', 'VERB':'VERB'
}

def norm_word(src_word):
    parsed = morph.parse(src_word)[0]
    tag = parsed.tag.POS
    if tag is None:
        return None
    word = parsed.normal_form
    if tag in tags_conversion:
        result = word + '_' + tags_conversion[tag]
        if result in w2v.vocab:
            return result
    for candidate in w2v_tags:
        result = word + '_' + candidate
        if result in w2v.vocab:
            return result
    return None

stops = set(stopwords.words("english")) | set(stopwords.words("russian"))

def text_to_words(text):
    text_alpha = re.sub("[^а-яА-Яa-zA-Z]"," ", text)
    words = text_alpha.lower().split()
    words = [w for w in words if not w in stops]
    words = [norm_word(w) for w in words]
    words = [w for w in words if w is not None]
    return ' '.join( words )

text_to_words('красивый дом')

'красивый_ADJ дом_NOUN'

In [126]:
data = pd.read_csv(file_prefix + '_split.csv', sep='\t')

data['norm_name'] = data['tender_name'].apply(text_to_words)
data['norm_lots'] = data['lots'].apply(text_to_words)
data['norm_okpd'] = data['okpd2_name'].apply(text_to_words)

data.head()

,tender_id,tender_name,lots,otrasl,podortasl,okpd2,okpd2_name,norm_name,norm_lots,norm_okpd
0,50000048,Филиал Федерального бюджетного учреждения здра...,Услуги по оценке собственности,"Бизнес, финансы, страхование, маркетинг и реклама","Бухгалтерский учет, оценка, аудит",683116120,Услуги посреднические при оценке нежилого недв...,филиал_NOUN федеральный_ADJ бюджетный_ADJ учре...,услуга_NOUN оценка_NOUN собственность_NOUN,услуга_NOUN посреднический_ADJ оценка_NOUN неж...
1,50000049,МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ САРГАТСКОГ...,Поставка газа,Топливо и энергетика,Газ,352000000,"Газы горючие искусственные, услуги по распреде...",муниципальный_ADJ унитарный_ADJ предприятие_NO...,поставка_NOUN газа_NOUN,газа_NOUN горючий_ADJ искусственный_ADJ услуга...
2,50000050,АКЦИОНЕРНАЯ КОМПАНИЯ 'АЛРОСА' (ПУБЛИЧНОЕ АКЦИО...,НИГП. Разработка минералого-геохимических крит...,"Строительство, недвижимость и архитектура","Проектные работы, геодезия, картография, инжен...",711239000,"Услуги в области геологических, геофизических ...",акционерный_ADJ компания_NOUN алрос_NOUN публи...,разработка_NOUN геохимический_ADJ критерий_NOU...,услуга_NOUN область_NOUN геологический_ADJ гео...
3,50000149,АКЦИОНЕРНОЕ ОБЩЕСТВО 'ТРАНСНЕФТЬ - ЦЕНТРАЛЬНАЯ...,Техническое обслуживание теплоэнергетического ...,"Офис, дом",Услуги,353012000,Услуги по снабжению паром и горячей водой по т...,акционерный_ADJ общество_NOUN транснефть_NOUN ...,технический_ADJ обслуживание_NOUN теплоэнергет...,услуга_NOUN снабжение_NOUN паром_NOUN горячий_...
4,50000153,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ЭНЕРГЕТИКИ И ЭЛ...,Открытый запрос предложений № 476-ОЗП/16-СМП н...,"Офис, дом",Мебель,310100000,Мебель для офисов и предприятий торговли,публичный_ADJ акционерный_ADJ общество_NOUN эн...,открытый_ADJ запрос_NOUN предложение_NOUN опре...,мебель_NOUN офис_NOUN предприятие_NOUN торговл...


In [127]:
data.to_csv(file_prefix + '_norm.csv', sep='\t', index=False, encoding='utf-8')